In [ ]:
import mujoco
import numpy as np
import mediapy as media


xml = """
<mujoco model="fixed_sphere_with_moving_body">
    <compiler angle="degree" coordinate="local"/>
    <option timestep="0.01" gravity="0 0 -9.81"/>
    
    <worldbody>
        <!-- Fixed sphere -->
        <body name="fixed_sphere" pos="0 0 0">
            <geom type="sphere" size="1" rgba="0.5 0.5 0.5 0.3" contype="0" conaffinity="0"/>
        </body>
        
        <!-- Moving body -->
        <body name="moving_body" pos="0 0 -1">
            <joint name="ball_joint" type="ball" pos="0 0 1" limited="true" range="0 90"/>
            <geom type="sphere" size="0.1" rgba="1 0 0 1"/>
        </body>
    </worldbody>
    
    <actuator>
        <motor joint="ball_joint" ctrlrange="-1 1" ctrllimited="true"/>
    </actuator>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# Simulation parameters
duration = 1 # simulation duration in seconds
time_step = model.opt.timestep
num_steps = int(duration / time_step)

# Initial force to apply
force_magnitude = 10.0

# Create a window to visualize the simulation
renderer = mujoco.Renderer(model)

camera = mujoco.MjvCamera()
camera.lookat = np.array([0, 0, 0])
camera.distance = 1
camera.elevation = -90
camera.azimuth = 90


# Get the index of the moving body
moving_body_id = model.body('moving_body').id

positions = []
frames = []

for step in range(num_steps):
    
    if step % 2 == 0:  # Apply force every 100 timesteps
        dir = np.random.uniform(-1,1,3)
        dir /= np.linalg.norm(dir)
        dir *= 5

        force = force_magnitude * dir
        data.xfrc_applied[moving_body_id, :3] = force  # Apply force in the x, y, z direction
        #print(f"Applied force {force} at timestep {step}")

    # Step the simulation
    mujoco.mj_step(model, data)

    # Extract the position of the moving body
    moving_body_position = data.xpos[moving_body_id]

    # if moving_body_position[2] > 0:
    #     moving_body_position[2] = 0
    #     data.qpos[moving_body_id*3+2] = 0

    positions.append(moving_body_position.copy())
    #print(f"Timestep {step}: Position of moving body: {moving_body_position}")

    # Render the scene
    renderer.update_scene(data)
    frame = renderer.render()
    frames.append(frame)

    
    #media.show_image(renderer.render())

    # Clear the applied force after application
    data.xfrc_applied[moving_body_id, :3] = 0

    # Slow down the simulation for visualization
    #time.sleep(time_step)

media.show_video(frames, fps=60)

# Close the renderer
renderer.close()

# Verify that positions are on the surface of the sphere
for pos in positions:
    distance_from_origin = np.linalg.norm(pos)
    assert np.isclose(distance_from_origin, 1.0, atol=1e-2), f"Body not on sphere surface: {pos}"
    #print(f"Verified position on sphere: {pos}")

# with mujoco.Renderer(model) as renderer:
#   mujoco.mj_forward(model, data)
#   renderer.update_scene(data)


#   media.show_image(renderer.render())

In [ ]:
outputs = model(**inputs, output_hidden_states = True, output_attentions = True)
last_hidden_state = outputs.hidden_states[-1].squeeze(0)
pre_c = model.pre_classifier(last_hidden_state)
classifier = model.classifier(pre_c).cpu().detach().numpy()